In [41]:
import os
from os import path

import numpy as np
import pandas as pd
from scipy import stats
from tabulate import tabulate

DDI_SCORES_PATH = "D:\\JavaTraining\\mgr\\saved_scores\\ddi"
columns = [
    "ONSBoost(base_estimator=GaussianNB(), n_estimators={}, protection_period=50, update_period=50, window_size=10)",
    "ONSBoost(base_estimator=GaussianNB(), n_estimators={}, protection_period=50, update_period=100, window_size=10)",
    "ONSBoost(base_estimator=GaussianNB(), n_estimators={}, protection_period=50, update_period=200, window_size=10)",
    "ONSBoost(base_estimator=GaussianNB(), n_estimators={}, protection_period=50, update_period=50, window_size=100)",
    "ONSBoost(base_estimator=GaussianNB(), n_estimators={}, protection_period=50, update_period=100, window_size=100)",
    "ONSBoost(base_estimator=GaussianNB(), n_estimators={}, protection_period=50, update_period=200, window_size=100)",
    "ONSBoost(base_estimator=GaussianNB(), n_estimators={}, protection_period=50, update_period=50, window_size=200)",
    "ONSBoost(base_estimator=GaussianNB(), n_estimators={}, protection_period=50, update_period=100, window_size=200)",
    "ONSBoost(base_estimator=GaussianNB(), n_estimators={}, protection_period=50, update_period=200, window_size=200)",
    "ONSBoost(base_estimator=GaussianNB(), n_estimators={}, protection_period=100, update_period=50, window_size=10)",
    "ONSBoost(base_estimator=GaussianNB(), n_estimators={}, protection_period=100, update_period=100, window_size=10)",
    "ONSBoost(base_estimator=GaussianNB(), n_estimators={}, protection_period=100, update_period=200, window_size=10)",
    "ONSBoost(base_estimator=GaussianNB(), n_estimators={}, protection_period=100, update_period=50, window_size=100)",
    "ONSBoost(base_estimator=GaussianNB(), n_estimators={}, protection_period=100, update_period=100, window_size=100)",
    "ONSBoost(base_estimator=GaussianNB(), n_estimators={}, protection_period=100, update_period=200, window_size=100)",
    "ONSBoost(base_estimator=GaussianNB(), n_estimators={}, protection_period=100, update_period=50, window_size=200)",
    "ONSBoost(base_estimator=GaussianNB(), n_estimators={}, protection_period=100, update_period=100, window_size=200)",
    "ONSBoost(base_estimator=GaussianNB(), n_estimators={}, protection_period=100, update_period=200, window_size=200)",
    "ONSBoost(base_estimator=GaussianNB(), n_estimators={}, protection_period=200, update_period=50, window_size=10)",
    "ONSBoost(base_estimator=GaussianNB(), n_estimators={}, protection_period=200, update_period=100, window_size=10)",
    "ONSBoost(base_estimator=GaussianNB(), n_estimators={}, protection_period=200, update_period=200, window_size=10)",
    "ONSBoost(base_estimator=GaussianNB(), n_estimators={}, protection_period=200, update_period=50, window_size=100)",
    "ONSBoost(base_estimator=GaussianNB(), n_estimators={}, protection_period=200, update_period=100, window_size=100)",
    "ONSBoost(base_estimator=GaussianNB(), n_estimators={}, protection_period=200, update_period=200, window_size=100)",
    "ONSBoost(base_estimator=GaussianNB(), n_estimators={}, protection_period=200, update_period=50, window_size=200)",
    "ONSBoost(base_estimator=GaussianNB(), n_estimators={}, protection_period=200, update_period=100, window_size=200)",
    "ONSBoost(base_estimator=GaussianNB(), n_estimators={}, protection_period=200, update_period=200, window_size=200)", ]

In [42]:
np.load("D:\\JavaTraining\\mgr\\saved_scores\\ddi\\5_w[0.2, 0.8]_s101010_nc250_nf10_cs200.npy").shape

(27, 249, 1)

In [43]:
def load_mean_scores(directory_path: str, n: int):
    saved_scores = [path.abspath(path.join(DDI_SCORES_PATH, file)) for file in os.listdir(directory_path) if file.startswith(str(n))]
    formatted_columns = [col.format(str(n)) for col in columns]
    all_data = pd.DataFrame()
    for scores_path in saved_scores:
        weights = scores_path.split('_w')[1].split('_')[0]
        seed = scores_path.split(']_s')[1].split('_')[0]
        mean_scores = np.load(scores_path).mean(axis=1)
        new_frame = pd.DataFrame(columns=formatted_columns, data=mean_scores.T)
        new_frame['stream'] = weights + "_" + seed
        all_data = pd.concat([all_data, new_frame])
    ensembles_parameters = [
        {
            'i': i,
            'n': n,
            'protection_period': int(clf.split('protection_period=')[1].split(',')[0]),
            'update_period': int(clf.split('update_period=')[1].split(',')[0]),
            'window_size': int(clf.split('window_size=')[1].split(')')[0]),
            'params': f"p{clf.split('protection_period=')[1].split(',')[0].rjust(2, '0')}_k{clf.split('update_period=')[1].split(',')[0]}_w{clf.split('window_size=')[1].split(')')[0]}"
        } for i, clf in enumerate(all_data.T.index.to_list()) if clf != 'stream'
    ]
    ensembles_parameters.sort(key=lambda parameters: (parameters['protection_period'], parameters['update_period'], parameters['window_size']))
    ordered_columns = [columns[ordered_v['i']].format(str(n)) for ordered_v in ensembles_parameters]
    return all_data.set_index('stream').T.reindex(ordered_columns).T, ensembles_parameters

In [44]:
pd_3_sorted, ensembles_params_3 = load_mean_scores(DDI_SCORES_PATH, 3)
pd_5_sorted, ensembles_params_5 = load_mean_scores(DDI_SCORES_PATH, 5)
pd_10_sorted, ensembles_params_10 = load_mean_scores(DDI_SCORES_PATH, 10)
pd_all_sorted = pd.concat([pd_3_sorted, pd_5_sorted, pd_10_sorted], axis=1)
pd_by_clf = pd_all_sorted.T.reset_index()
pd_by_clf.insert(1, 'n', pd_by_clf['index'].str.split('n_estimators=', expand=True)[1].str.split(',', expand=True)[0])
pd_by_clf.insert(2, 'k', pd_by_clf['index'].str.split('update_period=', expand=True)[1].str.split(',', expand=True)[0])
pd_by_clf.insert(3, 'p', pd_by_clf['index'].str.split('protection_period=', expand=True)[1].str.split(',', expand=True)[0])
pd_by_clf.insert(4, 'w', pd_by_clf['index'].str.split('window_size=', expand=True)[1].str.split(')', expand=True)[0])

pd_by_clf = pd_by_clf.drop(['index'], axis=1)
pd_by_clf = pd_by_clf.set_index(['n', 'k', 'p', 'w'])
pd_by_clf.to_csv('./pd_by_clf')

In [45]:
scores_n_3 = pd_3_sorted.T.scores_n_5()
scores_n_3.shape

(27, 12)

In [46]:
pd_by_clf

stream          [0.1, 0.9]_100000  [0.1, 0.9]_1000  [0.1, 0.9]_101010  \
n  k   p   w                                                            
3  50  50  10            0.741405         0.773666           0.776186   
           100           0.659596         0.698469           0.719366   
           200           0.654566         0.705318           0.701149   
   100 50  10            0.783066         0.790393           0.806441   
           100           0.697431         0.739530           0.770346   
...                           ...              ...                ...   
10 100 200 100           0.799782         0.774488           0.799788   
           200           0.797504         0.768226           0.787487   
   200 200 10            0.840507         0.802367           0.818190   
           100           0.815248         0.782193           0.807371   
           200           0.810189         0.785103           0.807889   

stream          [0.2, 0.8]_100000  [0.2, 0.8]_1000  [0.2, 0.8]_101010  \
n  k   p   w                                                            
3  50  50  10            0.796141         0.828110           0.838499   
           100           0.736033         0.776210           0.790106   
           200           0.732953         0.763836           0.787903   
   100 50  10            0.826458         0.849666           0.852088   
           100           0.748814         0.797254           0.797682   
...                           ...              ...                ...   
10 100 200 100           0.865144         0.855606           0.850757   
           200           0.861406         0.852764           0.846855   
   200 200 10            0.886212         0.874853           0.860065   
           100           0.872580         0.863676           0.860960   
           200           0.873020         0.861694           0.859743   

stream          [0.2, 0.9]_100000  [0.2, 0.9]_1000  [0.2, 0.9]_101010  \
n  k   p   w                                                            
3  50  50  10            0.790553         0.827773           0.836028   
           100           0.735337         0.777696           0.783994   
           200           0.737182         0.780837           0.787962   
   100 50  10            0.828966         0.846115           0.847724   
           100           0.758227         0.773759           0.815002   
...                           ...              ...                ...   
10 100 200 100           0.860873         0.856236           0.848525   
           200           0.859604         0.848768           0.848560   
   200 200 10            0.887787         0.873755           0.865314   
           100           0.876162         0.863191           0.861052   
           200           0.880821         0.862149           0.858592   

stream          [0.3, 0.6]_100000  [0.3, 0.6]_1000  [0.3, 0.6]_101010  
n  k   p   w                                                           
3  50  50  10            0.825432         0.839856           0.854881  
           100           0.765283         0.805507           0.807256  
           200           0.764946         0.806189           0.805384  
   100 50  10            0.842545         0.864351           0.867002  
           100           0.775055         0.809749           0.826018  
...                           ...              ...                ...  
10 100 200 100           0.886317         0.885019           0.874401  
           200           0.890837         0.884580           0.870848  
   200 200 10            0.903914         0.905366           0.881941  
           100           0.897380         0.896098           0.880336  
           200           0.900664         0.897131           0.879246  

[81 rows x 12 columns]

In [47]:
t_statistics_3 = np.zeros((len(columns), len(columns)))
p_value_3 = np.zeros((len(columns), len(columns)))
advantages_3 = np.zeros((len(columns), len(columns)))
stat_significance_3 = np.zeros((len(columns), len(columns)))
alpha = .1

for i in range(len(columns)):
    for j in range(len(columns)):
        t_statistics_3[i, j], p_value_3[i, j] = stats.ttest_ind(scores_n_3[i], scores_n_3[j])
        advantages_3[i, j] = t_statistics_3[i, j] > 0
        stat_significance_3[i, j] = p_value_3[i, j] <= alpha
stat_sig_diff_3 = stat_significance_3 * advantages_3

headers = [x['params'] for x in ensembles_params_3]

print(tabulate(np.concatenate(([[header] for header in headers], p_value_3), axis=1), headers, floatfmt='.2f'))

                  p50_k50_w10    p50_k50_w100    p50_k50_w200    p50_k100_w10    p50_k100_w100    p50_k100_w200    p50_k200_w10    p50_k200_w100    p50_k200_w200    p100_k50_w10    p100_k50_w100    p100_k50_w200    p100_k100_w10    p100_k100_w100    p100_k100_w200    p100_k200_w10    p100_k200_w100    p100_k200_w200    p200_k50_w10    p200_k50_w100    p200_k50_w200    p200_k100_w10    p200_k100_w100    p200_k100_w200    p200_k200_w10    p200_k200_w100    p200_k200_w200
--------------  -------------  --------------  --------------  --------------  ---------------  ---------------  --------------  ---------------  ---------------  --------------  ---------------  ---------------  ---------------  ----------------  ----------------  ---------------  ----------------  ----------------  --------------  ---------------  ---------------  ---------------  ----------------  ----------------  ---------------  ----------------  ----------------
p50_k50_w10              1.00            0.00       

In [48]:
ssig_pd_3 = pd.DataFrame(data=stat_sig_diff_3, columns=headers, index=headers)
ssig_pd_3

,p50_k50_w10,p50_k50_w100,p50_k50_w200,p50_k100_w10,p50_k100_w100,p50_k100_w200,p50_k200_w10,p50_k200_w100,p50_k200_w200,p100_k50_w10,...,p100_k200_w200,p200_k50_w10,p200_k50_w100,p200_k50_w200,p200_k100_w10,p200_k100_w100,p200_k100_w200,p200_k200_w10,p200_k200_w100,p200_k200_w200
p50_k50_w10,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
p50_k50_w100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
p50_k50_w200,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
p50_k100_w10,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,...,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0
p50_k100_w100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
p50_k100_w200,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
p50_k200_w10,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0
p50_k200_w100,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
p50_k200_w200,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
p100_k50_w10,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [50]:
ssig_pd_3.to_csv('ssigdiff_csv')

In [51]:
t_statistics_5 = np.zeros((len(columns), len(columns)))
p_value_5 = np.zeros((len(columns), len(columns)))
advantages_5 = np.zeros((len(columns), len(columns)))
stat_significance_5 = np.zeros((len(columns), len(columns)))
alpha = .1

scores_n_5 = pd_5_sorted.T.to_numpy()

for i in range(len(columns)):
    for j in range(len(columns)):
        t_statistics_5[i, j], p_value_5[i, j] = stats.ttest_ind(scores_n_5[i], scores_n_5[j])
        advantages_5[i, j] = t_statistics_5[i, j] > 0
        stat_significance_5[i, j] = p_value_5[i, j] <= alpha
stat_sig_diff_5 = stat_significance_5 * advantages_5

ssig_pd_5 = pd.DataFrame(data=stat_sig_diff_5, columns=headers, index=headers)
ssig_pd_5

,p50_k50_w10,p50_k50_w100,p50_k50_w200,p50_k100_w10,p50_k100_w100,p50_k100_w200,p50_k200_w10,p50_k200_w100,p50_k200_w200,p100_k50_w10,...,p100_k200_w200,p200_k50_w10,p200_k50_w100,p200_k50_w200,p200_k100_w10,p200_k100_w100,p200_k100_w200,p200_k200_w10,p200_k200_w100,p200_k200_w200
p50_k50_w10,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
p50_k50_w100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
p50_k50_w200,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
p50_k100_w10,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0
p50_k100_w100,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
p50_k100_w200,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
p50_k200_w10,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0
p50_k200_w100,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
p50_k200_w200,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
p100_k50_w10,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [52]:
ssig_pd_5.to_csv('./ssig_pd_5')